In [1]:
import torch
from torch import nn
from torch.utils.data.sampler import SubsetRandomSampler
from torch.nn import functional as F

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=12, stride=4, padding=0),
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU())
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 3, stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(9216, 4096),
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.fc1(out)
        out = F.adaptive_avg_pool2d(out.unsqueeze(1), output_size=1)
        return out

/opt/conda/lib/python3.8/site-packages/torch/_masked/__init__.py:223: UserWarning: Failed to initialize NumPy: module compiled against API version 0xf but this version of numpy is 0xd (Triggered internally at  /opt/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:68.)
  example_input = torch.tensor([[-3, -2, -1], [0, 1, 2]])


In [26]:
import numpy
import os

class GroundDataset(torch.utils.data.IterableDataset):

    def __init__(self, iterator):
        super(GroundDataset, self).__init__()
        self._iterator = iterator
        self._setup = False

    def _setup_iterator(self):
        self._iterator = self._iterator()

    def __iter__(self):
        if self._setup is False:
            self._setup_iterator()
            self._setup = True
        for x in self._iterator:
            yield x
            
def getItr(directory):

    def __iter__():
        for ds in os.listdir(directory):
            ds_name = os.path.join(directory,ds)
            data = None
            labels = None
            for f in os.listdir(ds_name):
                filename = os.path.join(ds_name,f)  
                if filename.find("data") > -1:
                    data = numpy.nan_to_num(numpy.load(filename),posinf=0,neginf=0)
                else:                   
                    labels = numpy.nan_to_num(numpy.load(filename),posinf=0,neginf=0)
                    print (filename,labels.mean())
            print(data.shape)        
            for i in range(data.shape[0]):  
                yield numpy.transpose(data[i].astype(numpy.float64),(2,1,0)),numpy.asarray([labels[i].flatten().astype(numpy.float64).mean()]).reshape((1,1))
    return __iter__              
            

In [3]:
learning_rate = 0.000005

model = AlexNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [35]:
num_epochs = 10
model_save_path = "."

iter_ = getItr("processed/train")
train_ds =  GroundDataset(iter_)
iter_ = getItr("processed/test")
test_ds = GroundDataset(iter_)
iter_ = getItr("processed/outumn")
val_ds = GroundDataset(iter_)

train_loss = nn.MSELoss()
dataloader =  torch.utils.data.DataLoader(train_ds, num_workers=0, batch_size = 64, shuffle=False)
valloader = torch.utils.data.DataLoader(val_ds, num_workers=0, batch_size = 64, shuffle=False)
testloader = torch.utils.data.DataLoader(test_ds, num_workers=0, batch_size = 64, shuffle=False)

In [34]:

total_step = 899#len(train_loader)

for epoch in range(num_epochs):
    for i, (data, labels) in enumerate(dataloader):  
        # Move tensors to the configured device
        data = data.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(data)
        loss = train_loss(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    # Validation
    with torch.no_grad():
        mse = []
        
        for data, labels in valloader:
            data = data.to(device)
            labels = labels.to(device)
            outputs = model(data)
            predicted = outputs.data
            mse.append(float(train_loss(labels.data,predicted))) 
            del data, labels, outputs
    
        print('MSE of the network on the {} validation samples: {} %'.format(5000, numpy.asarray(mse).mean())) 

processed/train/0/T36UXA_20180621T083651centinel2_nvdi_labels.npy 0.6065357
(2304, 228, 228, 3)
Epoch [1/10], Step [1/899], Loss: 0.0084
Epoch [1/10], Step [2/899], Loss: 0.0104
Epoch [1/10], Step [3/899], Loss: 0.0077
Epoch [1/10], Step [4/899], Loss: 0.0085
Epoch [1/10], Step [5/899], Loss: 0.0060
Epoch [1/10], Step [6/899], Loss: 0.0051
Epoch [1/10], Step [7/899], Loss: 0.0033
Epoch [1/10], Step [8/899], Loss: 0.0028
Epoch [1/10], Step [9/899], Loss: 0.0025
Epoch [1/10], Step [10/899], Loss: 0.0028
Epoch [1/10], Step [11/899], Loss: 0.0028
Epoch [1/10], Step [12/899], Loss: 0.0049
Epoch [1/10], Step [13/899], Loss: 0.0040
Epoch [1/10], Step [14/899], Loss: 0.0036
Epoch [1/10], Step [15/899], Loss: 0.0022
Epoch [1/10], Step [16/899], Loss: 0.0013
Epoch [1/10], Step [17/899], Loss: 0.0013
Epoch [1/10], Step [18/899], Loss: 0.0012
Epoch [1/10], Step [19/899], Loss: 0.0011
Epoch [1/10], Step [20/899], Loss: 0.0053
Epoch [1/10], Step [21/899], Loss: 0.0075
Epoch [1/10], Step [22/899], Lo

AttributeError: 'NoneType' object has no attribute 'shape'

In [18]:
with torch.no_grad():
    mse = []

    for data, labels in valloader:
        data = data.to(device)
        labels = labels.to(device)
        outputs = model(data)
        predicted = outputs.data
        mse.append(float(train_loss(labels.data,predicted))) 
        del data, labels, outputs

    print('MSE of the network on the {} test samples: {} %'.format(5000, numpy.asarray(mse).mean())) 

processed/validate/0/T36UXA_20180611T083601centinel2_nvdi_labels.npy 0.5701243
processed/validate/1/T37UCR_20180731T083601centinel2_nvdi_labels.npy 0.54268944
processed/validate/2/T37UCR_20180904T083549centinel2_nvdi_labels.npy 0.33022287
processed/validate/3/T37UCS_20180825T083549centinel2_nvdi_labels.npy 0.4912743
MSE of the network on the 5000 test samples: 0.019864353941456 %


In [31]:
with torch.no_grad():
    mse = []

    for data, labels in valloader:
        data = data.to(device)
        labels = labels.to(device)
        outputs = model(data)
        predicted = outputs.data
        mse.append(float(train_loss(labels.data,predicted))) 
        del data, labels, outputs

    print('MSE of the network on the {} test samples: {} %'.format(5000, numpy.asarray(mse).mean())) 

processed/spring/0/T36UXA_20180502T083601centinel2_nvdi_labels.npy 0.33934337
(2304, 228, 228, 3)
processed/spring/1/T36UXA_20180601T083651centinel2_nvdi_labels.npy 0.53730196
(2304, 228, 228, 3)
MSE of the network on the 5000 test samples: 0.032713217524966844 %


In [36]:
with torch.no_grad():
    mse = []

    for data, labels in valloader:
        data = data.to(device)
        labels = labels.to(device)
        outputs = model(data)
        predicted = outputs.data
        mse.append(float(train_loss(labels.data,predicted))) 
        del data, labels, outputs

    print('MSE of the network on the {} test samples: {} %'.format(5000, numpy.asarray(mse).mean())) 

processed/outumn/2/T36UXA_20180825T083549centinel2_nvdi_labels.npy 0.44718567
(2304, 228, 228, 3)
processed/outumn/3/T36UXA_20180904T083549centinel2_nvdi_labels.npy 0.34236798
(2304, 228, 228, 3)
MSE of the network on the 5000 test samples: 0.03517451195511967 %


In [28]:
for d in dataloader:
    print (d)

processed/train/0/T36UXA_20180621T083651centinel2_nvdi_labels.npy 0.6065357
(2304, 228, 228, 3)
[tensor([[[[ 830.,  890.,  842.,  ...,  906.,  932.,  956.],
          [ 846.,  856.,  846.,  ...,  946.,  924.,  966.],
          [ 854.,  842.,  866.,  ...,  964.,  928.,  942.],
          ...,
          [1016., 1010., 1096.,  ..., 1146., 1158., 1226.],
          [1046., 1002., 1056.,  ..., 1150., 1194., 1274.],
          [1084., 1042., 1076.,  ..., 1230., 1368., 1426.]],

         [[1380., 1430., 1386.,  ..., 1516., 1544., 1600.],
          [1386., 1408., 1392.,  ..., 1554., 1540., 1598.],
          [1396., 1376., 1436.,  ..., 1574., 1544., 1538.],
          ...,
          [1632., 1706., 1774.,  ..., 1698., 1678., 1686.],
          [1652., 1686., 1742.,  ..., 1680., 1682., 1686.],
          [1682., 1670., 1762.,  ..., 1696., 1776., 1748.]],

         [[1688., 1686., 1682.,  ..., 1754., 1776., 1826.],
          [1680., 1676., 1694.,  ..., 1772., 1732., 1768.],
          [1684., 1694., 1700

KeyboardInterrupt: 